<a href="http://colab.research.google.com/github/dipanjanS/nlp_workshop_odsc19/blob/master/Module05%20-%20NLP%20Applications/Project07B%20-%20Text%20Classification%20Deep%20Learning%20CNN%20Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A baseline for sentiment analysis

Sentiment Analysis can be viewed as a classification task that determines whether a text expresses a positive or negative opinion. 

In this notebook, we will train tradictional classifiers (such as SVM), for the task of sentiment analysis. 



In particular, we will train a model to classify the reviews about movies. 
There are several availabel datasets of movie reviews (for example, https://www.kaggle.com/youssefamdouni/movie-review-classification-using-spacy/data). 

However, in this tutorial, we will use the dataset provided by nltk:




In [ ]:
import nltk
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews

Let us to see some properties of this dataset

In [ ]:
print('Reviews:', len(movie_reviews.fileids()))
print('Labels:', movie_reviews.categories())
print('Number of words:', len(movie_reviews.words()) )
vocab=set(movie_reviews.words())
print('Size of the vocabulary (unique words):', len(vocab) )


The following cell rebuild the texts of the reviews. We will also save the labels for each review.

In [ ]:
X=[] #list to save the texts
y=[] #list to save the labels

for file_id in movie_reviews.fileids(): #this traverse all movides
    category=movie_reviews.categories(file_id)
    label=category[0]
    tokens=list(movie_reviews.words(file_id))
    text=' '.join(str(word) for word in tokens)
    X.append(text)
    y.append(label)

X=list(X)
print("# of reviews:", len(X),len(y))


As NLTK does not provide us any traning and test subsets for this dataset, we must create them. To do this, we will use the train_test_split method provided by sklearn. 

In [ ]:
# Splitting into training and testing sets
from sklearn.model_selection import train_test_split
#random_state: controls the shuffling applied to the data before applying the split. Pass an int for reproducible output across multiple function calls
#random_state=42 means that the method will always create the same particion
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)



Let's show the first review and its label from the training dataset:

In [ ]:
print(X_train[0])
print(y_train[0])

**Exercise: **

Shown five examples of negative instances in the training dataset:

In [ ]:
# write your code

### Encoding the labels


We could translate each class ("pos", and "neg") to 1 or 0 in the previous for.
However, sklearn already provides us a class to translate string classes to numbers or vectors. 

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
num_classes=2 # positive -> 1, negative -> 0
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)


**Exercise:**

Show some examples and their labels:

## Class distribution
We should always study which class distribution our dataset has

In [ ]:
from matplotlib import pyplot as plt

labels=['positive', 'negative']
num_positives = sum(y_train)
num_negatives = len(y_train) - num_positives

print('Number of positive and negative reviews in the training:',num_positives, num_negatives)

values=[num_positives, num_negatives]

plt.bar(labels, values)
plt.xlabel('Categories')
plt.ylabel('Number of instances')
plt.title('Class Distribution in the training dataset')
plt.show()

 **Exercise:**
How is the class distributon in the test dataset?. 
Please, instead of using a bar plot, show a pie chart.


In [ ]:
# write your code

## Training a SVM model

Now, we will use a SVM classifier. To represent the texts, we will use the tf-idf model. This model is implemented by sklearn using the classes CountVectorizer (which tokenizes the texts and represents them using the BoW models) and TfidfTransformer, which transforms the values produced by CountVectorizer to the tf-idf values

Once texts have been represented as vectors, we can train our SVM classifier. 
The sklearn library allows us to put togethere the three processes in a pipeline:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.pipeline import Pipeline
from sklearn import svm

pipeline = Pipeline([
    ('bow', CountVectorizer()), 
    ('tfidf', TfidfTransformer()),  
    ('classifier', svm.SVC())])

pipeline.fit(X_train,y_train)

print('pipeline (tf-idf + svm) was trained!!!')


## Evaluation

To evaluate our system, we measure metrics such as precision, recall and F1 on the test dataset (see Evaluation NLP tasks at https://github.com/isegura/MUIA2022/)

In [ ]:
from sklearn.metrics import classification_report
y_pred = pipeline.predict(X_test)

print("Baseline results using tf-idf (no cleaning) and svm")
print(classification_report(y_test, y_pred))


**Exercise:**

Explain the goal of the following function. What does it get?, and what does it return?


In [ ]:
import re
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
nltk.download('wordnet')

def clean_text(text):

    text=str(text).lower()

    tokens=nltk.word_tokenize(text)

    tokens = [PorterStemmer().stem(word) for word in tokens]
    
    p = re.compile('[a-zA-Z]+');
    filtered_tokens=[]
    for token in tokens:
        if len(token)>=3 and p.match(token):
            filtered_tokens.append(token)
            
    return filtered_tokens


Now, we can use this function to clean the texts. To do this, we can pass this function to the argument analyzer of the CountVectorizer object. We can also remove the stop words.

In [ ]:
from nltk.corpus import stopwords

pipeline = Pipeline([
    ('bow', CountVectorizer(stop_words=stopwords.words('english'), analyzer=clean_text)), 
    ('tfidf', TfidfTransformer()),  
    ('classifier', svm.SVC())])

pipeline.fit(X_train,y_train)

print('pipeline (tf-idf + svm) was trained!!!')


## Results by cleaning + tf-idf + svm

Does this approach improve the results?


In [ ]:
y_pred = pipeline.predict(X_test)
print("Baseline results using tf-idf (cleaning) and svm")
print(classification_report(y_test, y_pred))

**Exercise:**

The default kernel function of SVC is rbf https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html. Instead of using rbf, train the model using another kernel function (such as linear). Does it affect the performance?

In [ ]:
# write your code

**Exercise:**

Test other classifiers such as Logistic Regression or KNeighbors?

In [ ]:
#from sklearn.linear_model import LogisticRegression
